# Demo: Robust MNIST Classification
<hr />

This interactive notebook serves as a guide for training a robust MNIST classification network using the SABRE framework.

### Navigating to the Project Directory

In [ ]:
cd ..

## Installing dependencies

Run the following command to install dependencies from the `requirements.txt` file:

In [ ]:
pip install -r ./requirements.txt

Following the installation of dependencies, verify that PyTorch is correctly installed:

In [ ]:
import torch
print("Torch version:", torch.__version__)
print("GPU count:", torch.cuda.device_count())

## Running experiments


### Setup
Setting up the experiments involves preparing the environment by importing necessary libraries, defining key parameters, and loading the dataset.

**Imports**

To begin, import essential Python libraries and modules that will be used throughout our experiments. This includes standard libraries such as os for file and directory operations, matplotlib.pyplot for plotting, and torch.nn.functional for neural network operations.

In [ ]:
import os

root_path = "./"

from core.defenses.sabre import SabreWrapper
from core.attacks import EoTPGD

from datasets.mnist import get_mnist
from models.mnist import MNISTModel
from models.helpers import load_model
from experiments.utils import ceil_dec, sabre_train, evaluate
from experiments.setup import create_directories
from utils.log import printer

import matplotlib.pyplot as plt
import torch.nn.functional as F

This setup encompasses all necessary imports, from the core SABRE defense mechanism and EoT-PGD attack method, to utility functions that streamline the training and evaluation processes.

**Parameters**

Next, we define a set of parameters that will guide our experiments. These parameters capture everything from computational resource allocation (GPU or CPU) to the specifics of our model architecture and the adversarial context in which it will be trained and evaluated.

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

model_name = "SABRE"
epsilon = ceil_dec(80. / 255, 3)
batch_size = 32
n_variants = 10
normalize = False
use_rand = True
run_auto_attack = False
run_transfer_attacks = False

dataset_name = "mnist"
classes = ('zero', 'one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine')

n_channels = 1
learning_rate = 1e-2
plot_images_during_training = True

train_params = {
    "eps": epsilon,
    "n_channels": n_channels,
    "normalize": normalize,
    "learning_rate": learning_rate,
    "plot_images": plot_images_during_training
}

models_dir, logs_dir = create_directories(model_name, dataset_name)
records_file = os.path.join(logs_dir, "records.json")
traces_file = os.path.join(logs_dir, "traces.txt")
results_file = os.path.join(logs_dir, "results.txt")
model_path = f"{models_dir}/model.weights.pth"

Defining these parameters allows us to streamline the executions across different configuration scenarios, facilitating the comparison of results.

**Loading dataset**

We prepare both the training and validation datasets, and return DataLoaders with the specified batch size:

In [ ]:
data_root = os.path.join(root_path, "datasets")
train_data_loader, test_data_loader = get_mnist(batch_size=batch_size, data_root=data_root,
                                                train=True, val=True, return_loader=True)

With our setup complete, we are now ready to proceed to the core of our experiments: training and evaluating our models under various conditions to assess the effectiveness of the SABRE framework in enhancing model robustness against adversarial attacks.

### Model definition
To create robust models, we define our classifier architecture and provide it to SABRE, with its configuration parameters defined earlier:

In [ ]:
base_model = MNISTModel()
model = SabreWrapper(eps=epsilon, use_rand=use_rand, n_variants=n_variants, base_model=base_model)

### Training
During this phase, our goal is to develop a robust model capable of accurately classifying images from the CIFAR-10 dataset, even in the presence of adversarial attacks. Using our robust training function, we train the models over multiple epochs while meticulously logging progress, allowing us to monitor the model's evolving robustness and adjust parameters as needed to optimize performance.

In [ ]:
if os.path.exists(model_path):
    printer(f'Model already exists: {model_path}\n', traces_file)
else:
    printer(f'Training model...\n', traces_file)

    model = sabre_train(model, train_data_loader, n_epochs=100, learning_rate=train_params["learning_rate"],
                        save_path=model_path, log_path=traces_file, params=train_params)

### Evaluation
After training, the model undergoes a rigorous evaluation phase where it's exposed to various adversarial attack methods. In this instance, we assess the model's performance and robustness against the EoTPGD attack — a potent variant of the Projected Gradient Descent attack enhanced with Expectation over Transformation (EoT).

In [ ]:
model = load_model(model, model_path=model_path, log_path=traces_file)

attack = EoTPGD(eps=80./255, alpha=0.05, steps=7, eot_steps=10)
attack.set_model(model)

printer(f'Evaluating model... [Attack=EoT-PGD 7 steps]\n', traces_file)

evaluate(model, attack, test_data_loader, log_path=traces_file)

### Visualization
To visually assess the robustness of our model, we compare the original images with their adversarial counterparts and the model's reconstructions that are fed to the classifiers.

In [ ]:
images, labels = next(iter(test_data_loader))
images, labels = images.to(device), labels.to(device)

with torch.enable_grad():
    adversarial_images = attack(images, labels).float()

reconstructed_benign = model.preprocessing(images)
reconstructed_adversarial = model.preprocessing(adversarial_images)

outputs_benign = model.classify(images)
_, predicted_benign = outputs_benign.max(1)

outputs_robust_benign = model.classify(reconstructed_benign)
_, predicted_robust_benign = outputs_robust_benign.max(1)

outputs_adversarial = model.classify(adversarial_images)
_, predicted_adversarial = outputs_adversarial.max(1)

outputs_robust_adversarial = model.classify(reconstructed_adversarial)
_, predicted_robust_adversarial = outputs_robust_adversarial.max(1)

total = labels.size(0)
correct = predicted_robust_adversarial.eq(labels).sum().item()

printer(f"Robust Accuracy: {100 * correct / total:.2f}")

print("Plotting results for 5 images...")
for i in range(5):
    plt.figure(figsize=(8, 4))

    plt.subplot(1, 4, 1)
    plt.imshow(images[i].cpu().detach().squeeze().numpy())
    plt.title('Original')
    plt.axis('off')

    plt.subplot(1, 4, 2)
    plt.imshow(adversarial_images[i].cpu().detach().squeeze().numpy())
    plt.title('Adversarial')
    plt.axis('off')

    plt.subplot(1, 4, 3)
    plt.imshow(reconstructed_benign[i].cpu().detach().squeeze().numpy())
    plt.title('Reconstructed\nBenign')
    plt.axis('off')

    plt.subplot(1, 4, 4)
    plt.imshow(reconstructed_adversarial[i].cpu().detach().squeeze().numpy())
    plt.title('Reconstructed\nAdversarial')
    plt.axis('off')

    plt.show()

## Conclusion
This notebook, through the processes of training, evaluation, and visualization of MNIST within the context of adversarial robustness, effectively demonstrates the effectiveness of the SABRE framework and provides a high-level overview of of how it can be used to enhance the resilience of machine learning models against adversarial attacks.